In [5]:
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import json


In [6]:
import os
from pathlib import Path
import pinecone
from pprint import pprint

from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate

import google.generativeai as genai

In [3]:
PINECONE_API_KEY= "9ad6e8f9-0adb-4f03-b03b-2b94e650ecf0"

In [24]:
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name = model_name)

c:\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [23]:
pc = Pinecone(api_key=os.getenv(PINECONE_API_KEY))

dim = 768

# Create a Pinecone index
pc.create_index(
    name="rag-test",
    dimension=dim,
    metric="cosine",
       spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)
    

In [25]:
data = json.load(open("reviews.json"))

In [26]:
#create embedding for each review and store in vector db

processed_data = []
for reviews in data["professor_reviews"]: 
    #for review in reviews: 
        embedding = embeddings.embed_query(reviews["comment"])
        print(embedding)
        processed_data.append(
        {
            "values": embedding,
            "id": reviews["name"],
            "metadata":{
                "review": reviews["comment"],
                "subject": reviews["subject"],
                "stars": reviews["rating"],
            }
        }
        )

         

[-0.07956008613109589, -0.05713615193963051, -0.005552654154598713, 0.11396334320306778, -0.011716888286173344, 0.06675246357917786, 0.05737416073679924, -0.06933178007602692, -0.02216614969074726, 0.03438258171081543, 0.014162453822791576, 0.03880488872528076, -0.10654234886169434, 0.1311119794845581, 0.05655721575021744, -0.35569727420806885, -0.0023223247844725847, 0.18984311819076538, -0.09277277439832687, -0.03842540085315704, 0.07732058316469193, -0.1692458540201187, -0.01484204176813364, -0.01774458959698677, 0.007372279651463032, -0.17570070922374725, -0.15340280532836914, -0.03347885608673096, 0.2010720819234848, -0.09023763984441757, 0.04579569026827812, 0.051018182188272476, 0.027938323095440865, 0.14940455555915833, 0.08131767809391022, -0.05616965517401695, -0.0007164961425587535, 0.10761318355798721, -0.17868728935718536, 0.2200794219970703, 0.17454643547534943, 0.07324908673763275, 0.02532515674829483, 0.016513481736183167, -0.16305875778198242, 0.05251312628388405, 0.02

In [27]:
print(processed_data)

[{'values': [-0.07956008613109589, -0.05713615193963051, -0.005552654154598713, 0.11396334320306778, -0.011716888286173344, 0.06675246357917786, 0.05737416073679924, -0.06933178007602692, -0.02216614969074726, 0.03438258171081543, 0.014162453822791576, 0.03880488872528076, -0.10654234886169434, 0.1311119794845581, 0.05655721575021744, -0.35569727420806885, -0.0023223247844725847, 0.18984311819076538, -0.09277277439832687, -0.03842540085315704, 0.07732058316469193, -0.1692458540201187, -0.01484204176813364, -0.01774458959698677, 0.007372279651463032, -0.17570070922374725, -0.15340280532836914, -0.03347885608673096, 0.2010720819234848, -0.09023763984441757, 0.04579569026827812, 0.051018182188272476, 0.027938323095440865, 0.14940455555915833, 0.08131767809391022, -0.05616965517401695, -0.0007164961425587535, 0.10761318355798721, -0.17868728935718536, 0.2200794219970703, 0.17454643547534943, 0.07324908673763275, 0.02532515674829483, 0.016513481736183167, -0.16305875778198242, 0.05251312628

In [28]:
index = pc.Index("rag-test")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1"
)